In [21]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
import numpy as np 
import pandas as pd

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

### Import Data
Get dataset and split into individual sentences

Create word corpus (vocabulary list from dataset)

Calculate total number of unique words, +1 for the padding "words"

In [19]:
with open('data/captions.txt') as f:
    captions = [lines.strip().split(',') for lines in f.readlines()]

captions = captions[1:]
f.close()

In [30]:
captions = pd.DataFrame(captions)
captions.head(1)

,0,1,2,3,4,5,6
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,None,None,None,None,None


In [29]:
captions.describe()

,0,1,2,3,4,5,6
count,40455,40455,2447,715,64,5,1
unique,8091,39929,2288,705,64,5,1
top,1626754053_81126b67b6.jpg,"""Two dogs","""","and a backpack .""",sitting,"orange and red costumes for a parade .""",are waving to the crowd along the parade rout...
freq,5,37,19,3,1,1,1


In [46]:
idx = []
for i in [2,3,4,5,6]:
    idx += list(captions[captions[i].notnull()].index)

idx = list(set(idx))

for index in idx:
    captions.iloc[idx[0],:]

In [83]:
" ".join(captions.iloc[idx[0],[2,3,4,5,6]].fillna("").values)

' a very small dog jumps on an older man ."    '

In [ ]:
corpus = data.lower().split("\n")

tokenizer = Tokenizer()

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

In [ ]:
input_sequences = []
for line in corpus: # for each sentence in dataset
	token_list = tokenizer.texts_to_sequences([line])[0] # tokenize each sentence
	for i in range(1, len(token_list)): # create n-grams for each sentence so that
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences so that inputs will have same length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label (last word in sentence)
sequence, labels = input_sequences[:,:-1],input_sequences[:,-1]

labels = tf.keras.utils.to_categorical(labels, num_classes=total_words) #convert them to categorical and one hot encode them so that it can predict which word from the vocab it is

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(sequence, labels, epochs=100, verbose=1)
#print model.summary()
print(model)

In [ ]:
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

stuff

In [ ]:
    def get_discriminator(self):
        image_input = Input(shape=self.image_size)

        x = self.conv_block(image_input, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))
        x = self.conv_block(x, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))
        x = self.conv_block(x, 128, 4, 2, 'same', use_batch_norm=True, use_bias=False, activation=LeakyReLU(alpha=leaky_relu_slope))

        x = Flatten()(x)
        x = Dropout(0.4)(x)
        output_score = Dense(1)(x)

        return keras.Model(image_input, output_score, name="discriminator")